# Clasificación automatizada de cuentas con TFLearn

In [1]:
import nltk                                      # Natural Language processing. Para el análisis lintuistico de documentos. 
from nltk.stem.lancaster import LancasterStemmer # Permite transformar palabras para que sea mas entendible (letras de mas o errores de tipeo, ect).
stemmer = LancasterStemmer ()
import numpy
import tensorflow as tf
import tflearn                                   # Para la implementación de redes neuronales multicapa
import json
import random
import pickle                                    # Permite guardar el modelo y no cargarlo cada vez que se ejecute el programa
import csv
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


###         Carga de cuentas estándar desde archivo .json

In [2]:
with open("estandar.json", encoding='utf-8') as archivo:
    datos = json.load(archivo)
    
# print(datos) #muestra de datos y prueba de carga correcta

### Tokenizacion de cuentas y definición de parámetros

In [3]:
from tensorflow.python.framework import ops

# El 'try' permite almacenar los parametros del modelo para proximas ejecuciones. Se debe dejar comentado para las pruebas iniciales del sistema!!

########### Inicio comentado de almacenamiento ################################
# try: 
#     with open("variables.pickle","rb") as archivoPickle:
#         palabras, tags, entrenamiento, salida = pickle.load(archivoPickle)

# except:
###############################################################################

palabras=[]
tags=[]
auxX=[]  # palabras que se asignaran a la lista "palabras"
auxY=[]

for contenido in datos["cuentas"]:
    for patrones in contenido["patrones"]:
        auxPalabra = nltk.word_tokenize(patrones) # Toma una frase y la separa en palabras (tokens), para identificar los patrones a parametrizar
        palabras.extend(auxPalabra)               # Almacena todas las palabras tokenizadas 
        auxX.append(auxPalabra)                   
        auxY.append(contenido["tag"])

        if contenido["tag"] not in tags:          # Agrega un tag si no lo encuentra
            tags.append(contenido["tag"])

palabras = [stemmer.stem(w.lower()) for w in palabras if w != "\n"]   #stemmer permite identificar palabras equivalentes (por ejemplo: Banco = Bco. = banco)
palabras = sorted(list(set(palabras)))            # Se define la palabra
tags = sorted(tags)                               # Se define el tag

entrenamiento = []
salida = []
salidaVacia = [0 for _ in range(len(tags))]

for x, documento in enumerate(auxX):              # en X se guardara el indice de la palabra
    cubeta=[]                                     #Lista auxiliar para almacenar las palabras 
    auxPalabra = [stemmer.stem(w.lower()) for w in documento]
    for w in palabras:
        if w in auxPalabra:
            cubeta.append(1)
        else:
            cubeta.append(0)

    filaSalida = salidaVacia[:]
    filaSalida[tags.index(auxY[x])] = 1            #en la posicion del indice del tag, obtenemos el indice del auxiliar
    entrenamiento.append(cubeta)
    salida.append(filaSalida)

entrenamiento = numpy.array(entrenamiento)
salida = numpy.array(salida)

# ########### Inicio comentado de almacenamiento ##################################

#     with open("variables.pickle","wb") as archivoPickle:
#             pickle.dump((palabras, tags, entrenamiento, salida), archivoPickle)
            
# #################################################################################

### Entrenamiento de los datos
###### Se define el modelo a utilizar y los parámetros de entrenamiento

In [4]:

ops.reset_default_graph()


    ### Creación red neuronal inicial de 4 redes, con 10 neuronas c/u

red = tflearn.input_data(shape=[None,len(entrenamiento[0])]) # Datos de entrada a entrenar
red = tflearn.fully_connected(red,10)                        # Se crean capas conectadas con 10 unidades/neuronas
red = tflearn.fully_connected(red,10)
#red = tflearn.fully_connected(red,10)
red = tflearn.fully_connected(red,len(salida[0]),activation="softmax") # función de activación de las capas ocultas
red = tflearn.regression(red, optimizer='adam', loss='categorical_crossentropy') #permite obtener la probabilidad de que tan eficaz es la clasificacion de los tag

modelo = tflearn.DNN(red)
#try:
#    modelo.load("modelo.tflearn")
#except:

historial = modelo.fit(entrenamiento,salida,n_epoch=400,batch_size=40,show_metric=True) #Entrenamiento para el proceso de modelado

modelo.save("modelo.tflearn")


Training Step: 20799  | total loss: 1.72589 | time: 0.140s
| Adam | epoch: 400 | loss: 1.72589 - acc: 0.8426 -- iter: 2040/2076
Training Step: 20800  | total loss: 1.56481 | time: 0.146s
| Adam | epoch: 400 | loss: 1.56481 - acc: 0.8533 -- iter: 2076/2076
--
INFO:tensorflow:C:\Users\smart\Downloads\8_Chat\modelo.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:C:\Users\smart\Downloads\8_Chat\modelo.tflearn.data-00000-of-00001
INFO:tensorflow:200
INFO:tensorflow:C:\Users\smart\Downloads\8_Chat\modelo.tflearn.index
INFO:tensorflow:200
INFO:tensorflow:C:\Users\smart\Downloads\8_Chat\modelo.tflearn.meta
INFO:tensorflow:300


### Carga del balance a estandarizar (balance del cliente)

In [5]:
with open('balance_wesser.csv', newline='') as f:
    reader = csv.DictReader(f,delimiter=';')
    
    cuentas = []
    for row in reader: 
        cuentas.append(row["nombre_cliente"]) 
        #balance_salida.append(row)
        #print(row)




### Interfaz para pruebas Sistema - Usuario

In [7]:
    

def mainBot():
    respuesta = []  # Temporal !!!
    #entrada = input("Usuario: ")
    count = 1   ############### Provisorio
    for x in range(len(cuentas) - 5):
        #print("-------------------",count,"-------------------")       ############### Provisorio
        indice = cuentas[x]
        count = count + 1    ############### Provisorio
        cubeta = [0 for _ in range(len(palabras))]
        entradaProcesada = nltk.word_tokenize(indice)
        entradaProcesada = [stemmer.stem(palabra.lower()) for palabra in entradaProcesada]
        for palabraIndividual in entradaProcesada:
            for i,palabra in enumerate(palabras):
                if palabra == palabraIndividual:
                    cubeta[i] = 1
        resultados = modelo.predict([numpy.array(cubeta)])
        resultadosIndices = numpy.argmax(resultados)
        tag = tags[resultadosIndices]                  #Resultado de la busqueda de la cuenta estandar a la que pertenece

        for tagAux in datos["cuentas"]:
            if tagAux["tag"] == tag:
                respuesta = tagAux["respuestas"]
                
        print("Usuario: ",indice)
        print("Sistema: ",random.choice(respuesta))
        
mainBot()



Usuario:  CAJA
Sistema:  Disponible
Usuario:  BANCO CREDITO E INVERSIONES
Sistema:  Disponible
Usuario:  BCO.SANTANDER
Sistema:  Otros ingresos fuera de la explotación
Usuario:  BANCO DE CHILE
Sistema:  Disponible
Usuario:  DEPOSITOS A PLAZO
Sistema:  Depósitos a plazo
Usuario:  VARIOS DEUDORES
Sistema:  Deudores varios
Usuario:  ANTICIPOS A ACREEDORES
Sistema:  Otras reservas
Usuario:  PRESTAMOS AL PERSONAL
Sistema:  Anticipo y préstamos a los empleados
Usuario:  MATERIA PRIMA
Sistema:  Existencias, neto
Usuario:  PRODUCTOS EN PROCESO
Sistema:  Existencias, neto
Usuario:  PRODUCTOS TERMINADOS
Sistema:  Existencias, neto
Usuario:  SUBPRODUCTOS TERMINADOS
Sistema:  Gastos de administración y ventas 
Usuario:  SUMINISTROS DE FABRICACION
Sistema:  Otros ingresos fuera de la explotación
Usuario:  SUMINISTROS DE MANTENCION
Sistema:  Otros egresos fuera de la explotación 
Usuario:  P. P. M.
Sistema:  Ingresos de explotación
Usuario:  GASTOS DE CAPACITACION
Sistema:  Otros egresos fuera de la

### Interfaz Sistema-Usuario para pruebas de cuentas unitarias

In [8]:
def Prueb_unit():

    #entrada = input("Usuario: ")
    count = 1   ############### Provisorio
    while True:
        print("-------------------",count,"-------------------")       ############### Provisorio
        entrada = input("Usuario: ")
        #indice = cuentas[x]
        count = count + 1    ############### Provisorio
        cubeta = [0 for _ in range(len(palabras))]
        entradaProcesada = nltk.word_tokenize(entrada)
        entradaProcesada = [stemmer.stem(palabra.lower()) for palabra in entradaProcesada]
        for palabraIndividual in entradaProcesada:
            for i,palabra in enumerate(palabras):
                if palabra == palabraIndividual:
                    cubeta[i] = 1
        resultados = modelo.predict([numpy.array(cubeta)])
        resultadosIndices = numpy.argmax(resultados)
        tag = tags[resultadosIndices]

        for tagAux in datos["contenido"]:
            if tagAux["tag"] == tag:
                respuesta = tagAux["respuestas"]
        
        #print("Usuario: ",indice)
        print("Sistema: ",random.choice(respuesta))
        



------------------- 1 -------------------
Usuario: caja
Sistema:  disponible
------------------- 2 -------------------


KeyboardInterrupt: Interrupted by user